# FE017 - More features inverse distances and aggreagates

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [4]:
feature_files = [f for f in os.listdir('../data/FE016/')]

In [5]:
feature_files

['FE016-test-3JHH.parquet',
 'FE016-train-1JHC.parquet',
 'FE016-test-2JHH.parquet',
 'FE016-train-2JHC.parquet',
 'FE016-train-2JHN.parquet',
 'FE016-test-1JHN.parquet',
 'FE016-train-3JHC.parquet',
 'FE016-train-2JHH.parquet',
 'FE016-test-2JHC.parquet',
 'FE016-test-3JHC.parquet',
 'FE016-train-3JHH.parquet',
 'FE016-train-3JHN.parquet',
 'FE016-test-2JHN.parquet',
 'FE016-train-1JHN.parquet',
 'FE016-test-1JHC.parquet',
 'FE016-test-3JHN.parquet']

In [6]:
for f in feature_files:
    df = pd.read_parquet(f'../data/FE016/{f}')
    break

In [11]:
dist_cols = [x for x in df.columns if 'distance_' in x]

In [60]:
dist_to_0_cols = [x for x in dist_cols if 'to_0' in x]
dist_to_1_cols = [x for x in dist_cols if 'to_1' in x]

# Distance average min max and stddev

In [16]:
df['dist_to_0_mean'] = df[dist_to_0_cols].mean(axis=1)
df['dist_to_1_mean'] = df[dist_to_1_cols].mean(axis=1)
df['dist_to_0_min'] = df[dist_to_0_cols].min(axis=1)
df['dist_to_1_min'] = df[dist_to_1_cols].min(axis=1)
df['dist_to_0_max'] = df[dist_to_0_cols].max(axis=1)
df['dist_to_1_max'] = df[dist_to_1_cols].max(axis=1)
df['dist_to_0_std'] = df[dist_to_0_cols].std(axis=1)
df['dist_to_1_std'] = df[dist_to_1_cols].std(axis=1)

# Cube inverse of distances

In [36]:
for col in tqdm(dist_cols):
    df[col]
    temp = df[col]**3
    df[col+'_cube_inverse'] = temp**-1

100%|██████████| 54/54 [00:00<00:00, 136.40it/s]


# cube_inverse * mass, cube inverse * val

In [48]:
relative_col_names = [x.strip('distance')[1:] for x in dist_cols]

In [49]:
df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom0_atomic_mass,atom0_atomic_number,exact_mass_x,atom0_valence,atom0_spin_multiplicity,atom1_atomic_mass,atom1_atomic_number,exact_mass_y,atom1_valence,atom1_spin_multiplicity,left_middle_average_angle,right_middle_average_angle,distance,is_bond,closest_to_0,2nd_closest_to_0,3rd_closest_to_0,4th_closest_to_0,5th_closest_to_0,6th_closest_to_0,7th_closest_to_0,8th_closest_to_0,9th_closest_to_0,10th_closest_to_0,closest_to_1,2nd_closest_to_1,3rd_closest_to_1,4th_closest_to_1,5th_closest_to_1,6th_closest_to_1,7th_closest_to_1,8th_closest_to_1,9th_closest_to_1,10th_closest_to_1,is_closest_pair,distance_closest_to_0,is_bond_closest_to_0,distance_2nd_closest_to_0,is_bond_2nd_closest_to_0,distance_3rd_closest_to_0,is_bond_3rd_closest_to_0,distance_4th_closest_to_0,is_bond_4th_closest_to_0,distance_5th_closest_to_0,is_bond_5th_closest_to_0,distance_6th_closest_to_0,is_bond_6th_closest_to_0,distance_7th_closest_to_0,is_bond_7th_closest_to_0,distance_8th_closest_to_0,is_bond_8th_closest_to_0,distance_9th_closest_to_0,is_bond_9th_closest_to_0,distance_10th_closest_to_0,is_bond_10th_closest_to_0,distance_closest_to_1,is_bond_closest_to_1,distance_2nd_closest_to_1,is_bond_2nd_closest_to_1,distance_3rd_closest_to_1,is_bond_3rd_closest_to_1,distance_4th_closest_to_1,is_bond_4th_closest_to_1,distance_5th_closest_to_1,is_bond_5th_closest_to_1,distance_6th_closest_to_1,is_bond_6th_closest_to_1,distance_7th_closest_to_1,is_bond_7th_closest_to_1,distance_8th_closest_to_1,is_bond_8th_closest_to_1,distance_9th_closest_to_1,is_bond_9th_closest_to_1,distance_10th_closest_to_1,is_bond_10th_closest_to_1,closest_to_0_atomic_mass,closest_to_0_atomic_number,closest_to_0_exact_mass,closest_to_0_valence,closest_to_0_spin_multiplicity,2nd_closest_to_0_atomic_mass,2nd_closest_to_0_atomic_number,2nd_closest_to_0_exact_mass,2nd_closest_to_0_valence,2nd_closest_to_0_spin_multiplicity,3rd_closest_to_0_atomic_mass,3rd_closest_to_0_atomic_number,3rd_closest_to_0_exact_mass,3rd_closest_to_0_valence,3rd_closest_to_0_spin_multiplicity,4th_closest_to_0_atomic_mass,4th_closest_to_0_atomic_number,4th_closest_to_0_exact_mass,4th_closest_to_0_valence,4th_closest_to_0_spin_multiplicity,5th_closest_to_0_atomic_mass,5th_closest_to_0_atomic_number,5th_closest_to_0_exact_mass,...,angle_1_0_3rd0,angle_1_0_4th0,angle_1_0_5th0,angle_0_1_closest1,angle_0_1_2nd1,angle_0_1_3rd1,angle_0_1_4th1,angle_0_1_5th1,angle_1_0_6th0,angle_1_0_7th0,angle_1_0_8th0,angle_1_0_9th0,angle_1_0_10th0,angle_1_0_11th0,angle_1_0_6th1,angle_1_0_7th1,angle_1_0_8th1,angle_1_0_9th1,angle_1_0_10th1,angle_1_0_11th1,angle_0_closest0_1,angle_0_2nd0_1,angle_0_3rd0_1,angle_0_4th0_1,angle_0_5th0_1,angle_0_6th0_1,angle_0_7th0_1,angle_0_8th0_1,angle_0_9th0_1,angle_0_10th0_1,angle_0_11th0_1,angle_0_closest1_1,angle_0_2nd1_1,angle_0_3rd1_1,angle_0_4th1_1,angle_0_5th1_1,angle_0_6th1_1,angle_0_7th1_1,angle_0_8th1_1,angle_0_9th1_1,angle_0_10th1_1,angle_0_11th1_1,dist_to_0_mean,dist_to_1_mean,dist_to_0_min,dist_to_1_min,dist_to_0_max,dist_to_1_max,dist_to_0_std,dist_to_1_std,distance_closest_to_0_cube_inverse,distance_2nd_closest_to_0_cube_inverse,distance_3rd_closest_to_0_cube_inverse,distance_4th_closest_to_0_cube_inverse,distance_5th_closest_to_0_cube_inverse,distance_6th_closest_to_0_cube_inverse,distance_7th_closest_to_0_cube_inverse,distance_8th_closest_to_0_cube_inverse,distance_9th_closest_to_0_cube_inverse,distance_10th_closest_to_0_cube_inverse,distance_closest_to_1_cube_inverse,distance_2nd_closest_to_1_cube_inverse,distance_3rd_closest_to_1_cube_inverse,distance_4th_closest_to_1_cube_inverse,distance_5th_closest_to_1_cube_inverse,distance_6th_closest_to_1_cube_inverse,distance_7th_closest_to_1_cube_inverse,distance_8th_closest_to_1_cube_inverse,distance_9th_closest_to_1_cube_inverse,distance_10th_closest_to_1_cube_inverse,distance_12th_closest_to_0_cube_inverse,distance_13th_closest_to_0_cube_inverse,distance_14th_closest_to_0_cube_invers

In [52]:
for r in relative_col_names:
    df[f'{r}_atomic_mass_x_cube_inv_dist'] = df[f'distance_{r}_cube_inverse'] * df[f'{r}_atomic_mass']
    df[f'{r}_valence_x_cube_inv_dist'] = df[f'distance_{r}_cube_inverse'] * df[f'{r}_valence']
    df[f'{r}_spin_multiplicity_x_cube_inv_dist'] = df[f'distance_{r}_cube_inverse'] * df[f'{r}_spin_multiplicity']

# Bond Count
## This doesn't work because "is_bond" isn't correct - could it be a bug?

In [22]:
df = pd.read_parquet(f'../data/FE016/FE016-test-3JHH.parquet')
df['atom_0_bond_count'] = df[[x for x in df.columns if 'is_bond' in x and 'closest_to_0' in x]].sum(axis=1)
df['atom_1_bond_count'] = df[[x for x in df.columns if 'is_bond' in x and 'closest_to_1' in x]].sum(axis=1)

# Valence

In [34]:
valence_cols = [x for x in df.columns if '_valence' in x]
val_to_0_cols = [x for x in valence_cols if 'to_0' in x]
val_to_1_cols = [x for x in valence_cols if 'to_1' in x]

In [40]:
df['val_not_0_mean'] = df[val_to_0_cols].mean(axis=1)
df['val_not_1_mean'] = df[val_to_1_cols].mean(axis=1)
df['val_not_0_max'] = df[val_to_0_cols].max(axis=1)
df['val_not_1_max'] = df[val_to_1_cols].max(axis=1)
df['val_not_0_min'] = df[val_to_0_cols].min(axis=1)
df['val_not_1_min'] = df[val_to_1_cols].min(axis=1)
df['val_not_0_std'] = df[val_to_0_cols].std(axis=1)
df['val_not_1_std'] = df[val_to_1_cols].std(axis=1)

# Atomic Mass

In [41]:
atomic_mass_cols = [x for x in df.columns if '_atomic_mass' in x]
atomic_mass_to_0_cols = [x for x in atomic_mass_cols if 'to_0' in x]
atomic_mass_to_1_cols = [x for x in atomic_mass_cols if 'to_1' in x]

In [44]:
df['atomic_mass_not_0_mean'] = df[atomic_mass_to_0_cols].mean(axis=1)
df['atomic_mass_not_1_mean'] = df[atomic_mass_to_1_cols].mean(axis=1)
df['atomic_mass_not_0_max'] = df[atomic_mass_to_0_cols].max(axis=1)
df['atomic_mass_not_1_max'] = df[atomic_mass_to_1_cols].max(axis=1)
df['atomic_mass_not_0_min'] = df[atomic_mass_to_0_cols].min(axis=1)
df['atomic_mass_not_1_min'] = df[atomic_mass_to_1_cols].min(axis=1)
df['atomic_mass_not_0_std'] = df[atomic_mass_to_0_cols].std(axis=1)
df['atomic_mass_not_1_std'] = df[atomic_mass_to_1_cols].std(axis=1)

# Put it all together in a loop

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

feature_files = [f for f in os.listdir('../data/FE016/')]
for f in tqdm(feature_files):
    df = pd.read_parquet(f'../data/FE016/{f}')
    dist_cols = [x for x in df.columns if 'distance_' in x]
    dist_to_0_cols = [x for x in dist_cols if 'to_0' in x]
    dist_to_1_cols = [x for x in dist_cols if 'to_1' in x]
    df['dist_to_0_mean'] = df[dist_to_0_cols].mean(axis=1)
    df['dist_to_1_mean'] = df[dist_to_1_cols].mean(axis=1)
    df['dist_to_0_min'] = df[dist_to_0_cols].min(axis=1)
    df['dist_to_1_min'] = df[dist_to_1_cols].min(axis=1)
    df['dist_to_0_max'] = df[dist_to_0_cols].max(axis=1)
    df['dist_to_1_max'] = df[dist_to_1_cols].max(axis=1)
    df['dist_to_0_std'] = df[dist_to_0_cols].std(axis=1)
    df['dist_to_1_std'] = df[dist_to_1_cols].std(axis=1)
    
    df['dist_to_0_mean'] = df[dist_to_0_cols].mean(axis=1)
    df['dist_to_1_mean'] = df[dist_to_1_cols].mean(axis=1)
    df['dist_to_0_min'] = df[dist_to_0_cols].min(axis=1)
    df['dist_to_1_min'] = df[dist_to_1_cols].min(axis=1)
    df['dist_to_0_max'] = df[dist_to_0_cols].max(axis=1)
    df['dist_to_1_max'] = df[dist_to_1_cols].max(axis=1)
    df['dist_to_0_std'] = df[dist_to_0_cols].std(axis=1)
    df['dist_to_1_std'] = df[dist_to_1_cols].std(axis=1)
    
    df['dist_to_0_mean'] = df[dist_to_0_cols].mean(axis=1)
    df['dist_to_1_mean'] = df[dist_to_1_cols].mean(axis=1)
    df['dist_to_0_min'] = df[dist_to_0_cols].min(axis=1)
    df['dist_to_1_min'] = df[dist_to_1_cols].min(axis=1)
    df['dist_to_0_max'] = df[dist_to_0_cols].max(axis=1)
    df['dist_to_1_max'] = df[dist_to_1_cols].max(axis=1)
    df['dist_to_0_std'] = df[dist_to_0_cols].std(axis=1)
    df['dist_to_1_std'] = df[dist_to_1_cols].std(axis=1)
    
    valence_cols = [x for x in df.columns if '_valence' in x]
    val_to_0_cols = [x for x in valence_cols if 'to_0' in x]
    val_to_1_cols = [x for x in valence_cols if 'to_1' in x]
    df['val_not_0_mean'] = df[val_to_0_cols].mean(axis=1)
    df['val_not_1_mean'] = df[val_to_1_cols].mean(axis=1)
    df['val_not_0_max'] = df[val_to_0_cols].max(axis=1)
    df['val_not_1_max'] = df[val_to_1_cols].max(axis=1)
    df['val_not_0_min'] = df[val_to_0_cols].min(axis=1)
    df['val_not_1_min'] = df[val_to_1_cols].min(axis=1)
    df['val_not_0_std'] = df[val_to_0_cols].std(axis=1)
    df['val_not_1_std'] = df[val_to_1_cols].std(axis=1)
    
    atomic_mass_cols = [x for x in df.columns if '_atomic_mass' in x]
    atomic_mass_to_0_cols = [x for x in atomic_mass_cols if 'to_0' in x]
    atomic_mass_to_1_cols = [x for x in atomic_mass_cols if 'to_1' in x]
    df['atomic_mass_not_0_mean'] = df[atomic_mass_to_0_cols].mean(axis=1)
    df['atomic_mass_not_1_mean'] = df[atomic_mass_to_1_cols].mean(axis=1)
    df['atomic_mass_not_0_max'] = df[atomic_mass_to_0_cols].max(axis=1)
    df['atomic_mass_not_1_max'] = df[atomic_mass_to_1_cols].max(axis=1)
    df['atomic_mass_not_0_min'] = df[atomic_mass_to_0_cols].min(axis=1)
    df['atomic_mass_not_1_min'] = df[atomic_mass_to_1_cols].min(axis=1)
    df['atomic_mass_not_0_std'] = df[atomic_mass_to_0_cols].std(axis=1)
    df['atomic_mass_not_1_std'] = df[atomic_mass_to_1_cols].std(axis=1)

    for col in dist_cols:
        df[col]
        temp = df[col]**3
        df[col+'_cube_inverse'] = temp**-1
    relative_col_names = [x.strip('distance')[1:] for x in dist_cols]
    for r in relative_col_names:
        df[f'{r}_atomic_mass_x_cube_inv_dist'] = df[f'distance_{r}_cube_inverse'] * df[f'{r}_atomic_mass']
        df[f'{r}_valence_x_cube_inv_dist'] = df[f'distance_{r}_cube_inverse'] * df[f'{r}_valence']
        df[f'{r}_spin_multiplicity_x_cube_inv_dist'] = df[f'distance_{r}_cube_inverse'] * df[f'{r}_spin_multiplicity']
    # Save results
    df.to_parquet('../data/FE017/{}'.format(f.replace('FE016','FE017')))

In [49]:
df = pd.read_parquet('../data/FE017/FE017-test-1JHN.parquet')

In [50]:
[x for x in df.columns]

['id',
 'molecule_name',
 'atom_index_0',
 'atom_index_1',
 'type',
 'scalar_coupling_constant',
 'atom0_atomic_mass',
 'atom0_atomic_number',
 'exact_mass_x',
 'atom0_valence',
 'atom0_spin_multiplicity',
 'atom1_atomic_mass',
 'atom1_atomic_number',
 'exact_mass_y',
 'atom1_valence',
 'atom1_spin_multiplicity',
 'left_middle_average_angle',
 'right_middle_average_angle',
 'distance',
 'is_bond',
 'closest_to_0',
 '2nd_closest_to_0',
 '3rd_closest_to_0',
 '4th_closest_to_0',
 '5th_closest_to_0',
 '6th_closest_to_0',
 '7th_closest_to_0',
 '8th_closest_to_0',
 '9th_closest_to_0',
 '10th_closest_to_0',
 'closest_to_1',
 '2nd_closest_to_1',
 '3rd_closest_to_1',
 '4th_closest_to_1',
 '5th_closest_to_1',
 '6th_closest_to_1',
 '7th_closest_to_1',
 '8th_closest_to_1',
 '9th_closest_to_1',
 '10th_closest_to_1',
 'is_closest_pair',
 'distance_closest_to_0',
 'is_bond_closest_to_0',
 'distance_2nd_closest_to_0',
 'is_bond_2nd_closest_to_0',
 'distance_3rd_closest_to_0',
 'is_bond_3rd_closest_to_